In [2]:
# importations

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
%matplotlib tk

from copy import deepcopy

# suivre avancements boucles
from tqdm import tqdm

# parallelisation (mémoire partagée)
from multiprocess import Pool, cpu_count

# module d'éléments finis
from fem2D import *

# module de crétion de maillages simples (carrés et rectangles)
from simpleMeshes import *

nprocesses = cpu_count()
print(f"Code parallelized with {nprocesses} processes.")

Code parallelized with 12 processes.


In [3]:
# construction du maillage

# efface un potentiel maillage déjà chargé
eraseCurrentMesh()

N = 4       # ordre des éléments

# taille du milieu
Lx = 100000.0
Lz = 4*Lx
dx = Lx/5   # taille des éléments
elements, nodes = rectangleMesh(Lx,Lz,dx,N, botTop=False)
Δx = dx/(N+1)
nodesOrderChanged = False

dof = len(nodes)
print(f"Nombre de noeuds : {dof}")
print(f"Soit {2*dof} degré de liberté pour un champ 2D")

Nombre de noeuds : 2025
Soit 4050 degré de liberté pour un champ 2D


In [4]:
# plot des noeuds avec leur id

fig, ax = plt.subplots()
ax.set_aspect("equal", "box")

for n in nodes:
    ax.plot(n.x, n.y, "+k")
    ax.text(n.x, n.y, str(n.id), clip_on=True)

In [5]:
# plot des elements avec leur id

fig, ax = plt.subplots()
ax.set_aspect("equal", "box")

plotMeshLimits(elements, ax)

for e in elements:
    coords = e.getCoords()
    ax.text(np.mean(coords[:,0]), np.mean(coords[:,1]), str(e.id), clip_on=True)

In [6]:
plotMesh(elements, nodes, [2, 4])

<AxesSubplot:>

In [7]:
# Chargement des points/poids d'intégration de GLL + dérivée des interpolateurs lagrangiens sur ces points
xi,w,dh = readGLL(N)

In [8]:
# Pour chaque élément calcul:
# - du jacobien
# - de l'inverse de la matrice jacobienne
# On choicit l'interpolation bilinéaire comme bijection depuis un élément quelconque vers l'élément de référence

duphi = shapeFunctions["P1"]["duphi"]
dvphi = shapeFunctions["P1"]["dvphi"]

def computeJacobian(e):
        
    coords = e.getCoords()
    X, Y = coords[:,0], coords[:,1]
    
    Je11 = np.zeros((N+1,N+1))
    Je21 = np.zeros((N+1,N+1))
    Je12 = np.zeros((N+1,N+1))
    Je22 = np.zeros((N+1,N+1))
    
    e.detJ = np.zeros((N+1,N+1))
    
    e.iJe11 = np.zeros((N+1,N+1))
    e.iJe21 = np.zeros((N+1,N+1))
    e.iJe12 = np.zeros((N+1,N+1))
    e.iJe22 = np.zeros((N+1,N+1))

    dxdu = interp(duphi, X)
    dxdv = interp(dvphi, X)        
    dydu = interp(duphi, Y)
    dydv = interp(dvphi, Y)
    
    for i in range(N+1):
        for j in range(N+1):
            
            Je11[i,j] = dxdu(xi[i], xi[j])
            Je21[i,j] = dxdv(xi[i], xi[j])
            Je12[i,j] = dydu(xi[i], xi[j])
            Je22[i,j] = dydv(xi[i], xi[j])
            
            e.detJ[i,j] = Je11[i,j]*Je22[i,j] - Je21[i,j]*Je12[i,j]
            
            e.iJe11[i,j] =  Je22[i,j]/e.detJ[i,j]
            e.iJe22[i,j] =  Je11[i,j]/e.detJ[i,j]
            e.iJe21[i,j] = -Je21[i,j]/e.detJ[i,j]
            e.iJe12[i,j] = -Je12[i,j]/e.detJ[i,j]
            
    return e

print("Calcul du jacobien et de l'inverse de la matrice jacobienne:")
with Pool(nprocesses) as p:
    Element.id = 0
    elements = list(tqdm(p.imap(computeJacobian, elements), total=len(elements)))

Calcul du jacobien et de l'inverse de la matrice jacobienne:


100%|██████████| 120/120 [00:00<00:00, 499.50it/s]


In [9]:
# définition des paramètres mécaniques (béton)

E = 30e9        # module d'Young
ν = 0.25        # coefficient de Poisson

# paramètres de Lamé
λ = (E*ν) / ((1+ν)*(1-2*ν))
μ = E / (2*(1+ν))

# masse volumique
ρ = 2500.0

# vitesses des ondes
vp = np.sqrt((λ+2*μ)/ρ)
vs =np.sqrt(μ/ρ)

print(f"Vitesse des ondes P (pressure) : {vp/1000:.3f}km/s")
print(f"Vitesse des ondes S (shear) : {vs/1000:.3f}km/s")

Vitesse des ondes P (pressure) : 3.795km/s
Vitesse des ondes S (shear) : 2.191km/s


In [10]:
# calcul des matrices élémentaires

δ = np.eye(N+1)

def computeStiffnes(e):
    
    # KeTemp = np.zeros((N+1,N+1,N+1,N+1))
    e.Ke = np.zeros((2*(N+1)**2,2*(N+1)**2))
    
    Kdxdx = np.zeros((N+1,N+1,N+1,N+1))
    Kdydy = np.zeros((N+1,N+1,N+1,N+1))
    Kdxdy = np.zeros((N+1,N+1,N+1,N+1))
    Kdydx = np.zeros((N+1,N+1,N+1,N+1))
    
    for a in range(N+1):
        for b in range(N+1):
            for c in range(N+1):
                for d in range(N+1):
                       
                        for p in range(N+1):
                            for q in range(N+1):
                                
                                Kdxdx[a,b,c,d] += w[p]*w[q] * (e.iJe11[p,q]*dh[a,p]*δ[b,q] + e.iJe12[p,q]*dh[b,q]*δ[a,p])*(e.iJe11[p,q]*dh[c,p]*δ[d,q] + e.iJe12[p,q]*dh[d,q]*δ[c,p]) * e.detJ[p,q]
                                Kdydy[a,b,c,d] += w[p]*w[q] * (e.iJe21[p,q]*dh[a,p]*δ[b,q] + e.iJe22[p,q]*dh[b,q]*δ[a,p])*(e.iJe21[p,q]*dh[c,p]*δ[d,q] + e.iJe22[p,q]*dh[d,q]*δ[c,p]) * e.detJ[p,q]
                                Kdxdy[a,b,c,d] += w[p]*w[q] * (e.iJe11[p,q]*dh[a,p]*δ[b,q] + e.iJe12[p,q]*dh[b,q]*δ[a,p])*(e.iJe21[p,q]*dh[c,p]*δ[d,q] + e.iJe22[p,q]*dh[d,q]*δ[c,p]) * e.detJ[p,q]
                                Kdydx[a,b,c,d] += w[p]*w[q] * (e.iJe21[p,q]*dh[a,p]*δ[b,q] + e.iJe22[p,q]*dh[b,q]*δ[a,p])*(e.iJe21[p,q]*dh[c,p]*δ[d,q] + e.iJe22[p,q]*dh[d,q]*δ[c,p]) * e.detJ[p,q]
                                
    for i in range(2*(N+1)**2):
        for j in range(2*(N+1)**2):
            
            a,b = i//2//(N+1), i//2%(N+1)
            c,d = j//2//(N+1), j//2%(N+1)
            
            if i%2 == 0 and j%2 == 0:
                
                e.Ke[i,j] = (2*λ+μ)*Kdxdx[a,b,c,d] + μ*Kdydy[a,b,c,d]             
                
            if i%2 == 0 and j%2 != 0:
                
                e.Ke[i,j] = λ*Kdxdy[a,b,c,d] + μ*Kdydx[a,b,c,d]      
            
            if i%2 != 0 and j%2 == 0:
                
                e.Ke[i,j] = λ*Kdydx[a,b,c,d] + μ*Kdxdy[a,b,c,d]             
                
            if i%2 != 0 and j%2 != 0:
                
                e.Ke[i,j] = (2*λ+μ)*Kdydy[a,b,c,d] + μ*Kdxdx[a,b,c,d] 
                                
    return e

# elements = list(tqdm(map(computeStiffnes, elements), total=len(elements)))

print("Calcul des matrices de rigidité (stiffness) élémentaires:")
with Pool(nprocesses) as p:
    Element.id = 0
    elements = list(tqdm(p.imap(computeStiffnes, elements), total=len(elements)))

Calcul des matrices de rigidité (stiffness) élémentaires:


100%|██████████| 120/120 [00:02<00:00, 42.72it/s]


In [11]:
print("Calcul des matrices de masse élémentaires:")
for e in tqdm(elements):
    
    e.Me = np.zeros(2*(N+1)**2) # diagonal
    
    for i in range(2*(N+1)**2):
        e.Me[i] = ρ * w[i//2//(N+1)]*w[i//2%(N+1)]*e.detJ[i//2//(N+1),i//2%(N+1)]

Calcul des matrices de masse élémentaires:


100%|██████████| 120/120 [00:00<00:00, 16128.84it/s]


In [12]:
# assemblage

print("Assemblage des matrices globales K et M:")

K = np.zeros((2*dof,2*dof))
M = np.zeros(2*dof)

for e in tqdm(elements):
    for (i,n1) in enumerate(e.nodes):
        M[2*n1.id] += e.Me[2*i]
        M[2*n1.id+1] += e.Me[2*i+1]
        for (j,n2) in enumerate(e.nodes):
            K[2*n1.id,2*n2.id] += e.Ke[2*i,2*j]
            K[2*n1.id+1,2*n2.id] += e.Ke[2*i+1,2*j]
            K[2*n1.id,2*n2.id+1] += e.Ke[2*i,2*j+1]
            K[2*n1.id+1,2*n2.id+1] += e.Ke[2*i+1,2*j+1]

Assemblage des matrices globales K et M:


100%|██████████| 120/120 [00:00<00:00, 551.21it/s]


In [13]:
# définition source : sinus en cisaillement au niveau du sol : cond Dirichlet
# on note les noeuds appartenant à la base du rectangle

nodesBotId = []
for i,n in enumerate(nodes):
    if n.region == 2:
        nodesBotId.append(n.id)
        
nodesId = np.array([n.id for n in nodes])

In [14]:
# on déplace les noeuds en condition de Dirichlet en haut : permutations des lignes et les colonnes

if nodesOrderChanged: print("Nodes order already changed")
nodesOrderChanged = True

for targetRow,id in enumerate(nodesBotId):
    
    actualRow = np.where(nodesId == id)[0][0]
    
    # print(f"Nodes Id : {id} ; AR = {actualRow} ; TR = {targetRow}")
        
    # direction 1
    K[:,2*targetRow],K[:,2*actualRow] = K[:,2*actualRow],K[:,2*targetRow] # permutationlignes
    K[2*targetRow,:],K[2*actualRow,:] = K[2*actualRow,:],K[2*targetRow,:] # permutation colonnes
    K[2*targetRow, 2*targetRow],K[2*actualRow, 2*actualRow] = K[2*actualRow, 2*actualRow],K[2*targetRow, 2*targetRow]
    M[2*targetRow],M[2*actualRow] = M[2*actualRow],M[2*targetRow]
    # direction 2
    K[:,2*targetRow+1],K[:,2*actualRow+1] = K[:,2*actualRow+1],K[:,2*targetRow+1] # permutation lignes
    K[2*targetRow+1,:],K[2*actualRow+1,:] = K[2*actualRow+1,:],K[2*targetRow+1,:] # permutation colonnes
    K[2*targetRow+1, 2*targetRow+1],K[2*actualRow+1, 2*actualRow+1] = K[2*actualRow+1, 2*actualRow+1],K[2*targetRow+1, 2*targetRow+1]
    M[2*targetRow+1],M[2*actualRow+1] = M[2*actualRow+1],M[2*targetRow+1]
    # permutation des noeuds : facilite affichage résultats
    nodes[targetRow],nodes[actualRow] = nodes[actualRow],nodes[targetRow]

In [15]:
kmax = max(K.max(), -K.min())
plt.imshow(K, cmap="seismic", vmin=-kmax, vmax=kmax)

In [16]:
# inversion de la matrice de masse
iM = 1/M

In [17]:
# résolution temporelle

Nit = 500       # nombre d'itérations
cfl = 0.25      # critère de CFL
Δt = Δx/vp* cfl # calcul du dt tel que vérfiant le critère de CFL (vp vitesse max)
T = Δt*Nit      # temps total de simulation

U = np.zeros((Nit, 2*dof))  # initialisation du vecteur de dépalcements

time = np.linspace(0,T,Nit)
f0 = vs/(5*Δx)  # fréquence de la pertubation à la base

In [18]:
# tests sur valeurs paramètres
print(f"CFL = {Δt/Δx*vp:.4g}")
print(f"Longeur d'onde pertubation grande devant distance internoeuds : Δx = {Δx}m while λmin ~ {vs/(4*f0)}")

CFL = 0.25
Longeur d'onde pertubation grande devant distance internoeuds : Δx = 4000.0m while λmin ~ 5000.0


In [19]:
U[:,:] = 0.0

nbDofBot = 2*len(nodesBotId) # nombre de degré de libertés affectés par la condition de Dirichlet

print("Résolution dynamique:")
for i in tqdm(range(2,Nit)):
    # condition Dirichlet
    U[i,1:nbDofBot:2] = 0.0
    U[i,:nbDofBot:2] = np.sin(f0*(i-2)*Δt)
    # résolution dynamique par différences finies
    U[i,nbDofBot:] = 2*U[i-1,nbDofBot:] - U[i-2,nbDofBot:] - Δt**2*np.multiply(iM , K@U[i-1,:])[nbDofBot:]


Résolution dynamique:


100%|██████████| 498/498 [00:02<00:00, 247.94it/s]


In [20]:
def reorderOutput(U, nodesIdAtBeginning, nodes):
    U1 = deepcopy(U)
    nodesId = np.array([n.id for n in nodes])
    for targetRow,id in enumerate(nodesIdAtBeginning):
        actualRow = np.where(nodesId == id)[0][0]
        U1[:,2*targetRow], U1[:,2*actualRow] = U1[:,2*actualRow], U1[:,2*targetRow]
        U1[:,2*targetRow+1], U1[:,2*actualRow+1] = U1[:,2*actualRow+1], U1[:,2*targetRow+1]
    return U1

In [21]:
fig, ax = plt.subplots()
ax.set_aspect("equal", "box")

it = int(Nit/3)

U_reorder = reorderOutput(U, nodesBotId, nodes)

plotDeformedMesh(elements, U_reorder[it,:], ax, 1e4)

In [28]:
fig, ax = plt.subplots()
ax.set_aspect("equal", "box")

displ = U[:,::2]

# dmin = min(dtot.min(), -dtot.max())/3
dmin = 0.1

dots = ax.scatter([n.x for n in nodes], [n.y for n in nodes], marker=".",c=displ[0,:], zorder=4, cmap="seismic", vmin=dmin, vmax=-dmin)

cbar = plt.colorbar(dots)
ttl = ax.annotate("it = 0", xy=(0.05, 0.05), xycoords='axes fraction', bbox=dict(facecolor='white', edgecolor='grey'), zorder=5)

dit = 5

def animate(i):
    dots.set_array(displ[dit*i,:])
    cbar.update_normal(dots)
    ttl.set_text(f"it = {dit*i}")
    return dots,ttl,

ani = animation.FuncAnimation(fig, animate, interval=200, blit=True, frames = int(Nit/dit))

ani.event_source.start()
plt.show()

# enregistrement de l'animation

# writer = animation.PillowWriter(fps=15) 
# file = "../data/instabOndeElastiques.gif"

# writer = animation.FFMpegWriter(fps=15) 
# file = "../data/instabOndeElastiques.mp4"

# ani.save(file, writer=writer)

In [23]:
end = Nit
n_id = 3

plt.plot(time[:end], U[:end, 0], "k", lw = 2, label="source")

n_index = np.where(nodesId == n_id)[0][0]
plt.plot(time[:end], U[:end, 2*n_index], label=str(n_id-i))


plt.legend()

In [24]:
end = 1000

plt.plot(time[:end], np.sin(f0*time[:end]), "k", lw=3)
for i in range(len(nodesBotId)):
    plt.plot(time[:end], U[:end,2*i+1], "+")